## Doing imports

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib
from ipywidgets import interactive, fixed, widgets
matplotlib.rcParams['figure.figsize'] = (20.0, 20.0)

images_path = './images/'

Dithering consist in reducing color palette of the image allowing to use only x number of colors.

Here I'm implementig dithering using Floyd–Steinberg algorithm.

https://en.wikipedia.org/wiki/Floyd%E2%80%93Steinberg_dithering


### Lets define quatization function
The quantization function will take one channel and return it quantised or binarized

In [2]:
def quatiseAndAjust(image, numColors):
    ## To begin with i'll consider only binarization
    quantizedImage = np.copy(image)
    row,column = image.shape
    ## Diffusion coeficiens
    d1 = 7.0/16.0 ## Right pixel
    d2 = 3.0/16.0 ## Left-bottom pixel
    d3 = 5.0/16.0 ## Bottom pixel
    d4 = 1.0/16.0 ## Right-bottom pixel
    ##print(quantizedImage)
    for x in range(0, row-1):
        for y in range(0, column-1):
            oldPix = quantizedImage[x,y]
            newPix = np.ceil((np.floor((oldPix/255)*numColors)) / numColors*255)
            quantizedImage[x,y] = newPix
            error = oldPix - newPix
            ##print(x, y, quantizedImage[x,y], error)
            quantizedImage[x,y+1] += error*d1
            quantizedImage[x+1,y-1] += error*d2
            quantizedImage[x+1,y] += error*d3
            quantizedImage[x+1,y+1] += error*d4
            ##print(quantizedImage)
            ##print()
    

    # out = np.clip(quantizedImage,0,255)[0:row-1,0:column-1]
    
    return quantizedImage

Now lets define function for dithering

In [5]:
def dither(imageIn, numColors):
    image = cv2.imread(images_path + imageIn, -1) ## Loading BGR image
    gray=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)    
    red = quatiseAndAjust(image[:,:,0],numColors)
    green = quatiseAndAjust(image[:,:,1],numColors)
    blue = quatiseAndAjust(image[:,:,2],numColors)
    ## merging back colors
    outImage = cv2.merge((red, green, blue))
    
    ## Creating gray image
    gray = quatiseAndAjust(gray[:,:],numColors)
    
    plt.subplot(3,1,1)
    plt.imshow(image)
    plt.title('Original image')    
    
    plt.subplot(3,1,2)
    plt.imshow(outImage)
    plt.title('Dithered image')
    
    plt.subplot(3,1,3)
    plt.imshow(gray, cmap='gray')
    plt.title('Dithered gray image')
    
    plt.show()
    
    
    cv2.imwrite(images_path + 'out.jpg', cv2.cvtColor(outImage, cv2.COLOR_RGB2BGR))
    
    return outImage

try 'landscape_2.jpg' or 'car_plate_3.jpg'

In [6]:
interactive(dither, imageIn=fixed('car_plate_3.jpg'), numColors=(2, 16, 2))

interactive(children=(IntSlider(value=8, description='numColors', max=16, min=2, step=2), Output()), _dom_clas…

Inspired by this video https://www.youtube.com/watch?v=lseR6ZguBNY

Ciao